In [2]:
import multiset

class field(object):
    def __init__(self, symbol, massDim, lorentz_rank, spinor_rank, spinor_rank_conj):
        self.symbol = symbol #string symbol for field
        self.lorentz_rank = lorentz_rank #int indicating lorentz rank of field
        self.spinor_rank = spinor_rank #int indicating spinor rank of field
        self.spinor_rank_conj = spinor_rank_conj #int indicating spinor rank of field
        self.massDim= massDim #int indicating mass dimension of field
    def info(self):
        return 'symbol: ' + str(self.symbol) \
            + ' massDim: ' + str(self.massDim) \
            + ' lorentz_rank: ' + str(self.lorentz_rank) \
            + ' spinor_rank: ' + str(self.spinor_rank)
    def get_symbol(self):
        return self.symbol
    def get_lorentz_rank(self):
        return self.lorentz_rank
    def get_spinor_rank(self):
        return self.spinor_rank
    def get_spinor_rank_conj(self):
        return self.spinor_rank_conj
    def get_massDim(self):
        return self.massDim
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq
    
class term(object):
    def __init__(self, fields, lorentz_contractions):
        self.fields = {}
        self.contractions = Multiset()
    def add_field(self, new_field):
        

In [3]:
D = field('D', 1, 1, 0, 0)
F = field('F', 2, 2, 0, 0)
S = field('Pb_S_P', 3, 0, 0, 0)
V = field('Pb_V_P', 3, 1, 0, 0)
T = field('Pb_T_P', 3, 2, 0, 0)
Vp = field('Pb_Vp_P', 3, 1, 0, 0)
Sp = field('Pb_Sp_P', 3, 0, 0, 0)

In [4]:
# field_combo indicates how many of each field are contained in the term in question
field_dict = {'D': 2, 'F': 1, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
print(field_dict.keys())
print(field_dict.values())

dict_keys(['D', 'F', 'Pb_S_P', 'Pb_V_P', 'Pb_T_P', 'Pb_Vp_P', 'Pb_Sp_P'])
dict_values([2, 1, 0, 0, 0, 0, 0])


In [5]:
list(field_dict.values())

[2, 1, 0, 0, 0, 0, 0]

In [6]:
def convert_symbolList_to_fieldList(symbol_list):
    D = field('D', 1, 1, 0, 0)
    F = field('F', 2, 2, 0, 0)
    S = field('Pb_S_P', 3, 0, 0, 0)
    V = field('Pb_V_P', 3, 1, 0, 0)
    T = field('Pb_T_P', 3, 2, 0, 0)
    Vp = field('Pb_Vp_P', 3, 1, 0, 0)
    Sp = field('Pb_Sp_P', 3, 0, 0, 0)
    
    field_list = []
    for i in range(len(symbol_list)):
        if symbol_list[i] == 'D':
            field_list.append(D)
        if symbol_list[i] == 'F':
            field_list.append(F)
        if symbol_list[i] == 'Pb_S_P':
            field_list.append(S)
        if symbol_list[i] == 'Pb_V_P':
            field_list.append(V)
        if symbol_list[i] == 'Pb_T_P':
            field_list.append(T)
        if symbol_list[i] == 'Pb_Vp_P':
            field_list.append(Vp)
        if symbol_list[i] == 'Pb_Sp_P':
            field_list.append(Sp)
        
    return field_list


def lorentzRanks_list(field_dict):
    symbol_list = list(field_dict.keys())
    field_types = convert_symbolList_to_fieldList(symbol_list)
    numFields_list = list(field_dict.values())
    lor_ranks_list = [field_type.get_lorentz_rank() for field_type in field_types]
    N_fieldTypes = len(symbol_list)
    lorentzRanks = [num_fields*lor_rank for num_fields,lor_rank  in zip(numFields_list,lor_ranks_list)]
    # if field is 'F', 'T', number of lorentz indices is twice the number of fields
    #for i in range(N_fieldTypes):
    #    if symbol_list[i] == 'F' or symbol_list[i] == 'Pb_T_P':
    #        lorentzRanks[i] *= 2
    return lorentzRanks

  
def generate_lorentz_contractions(lorentzRanks_list, i0):
    # EXPLANATION: recursively generateS all ways of lorentz contracting fields. generate a set of sets of contractions. all sets of
    # contractions should have the same number of contractions. take first index in lorentzRanks_list and contract it with every field with non-zero lorentz rank. 
    # for each such contraction, update the lorentzRanks_list by decrementing the lorentz rank of each contracted
    # field by one for each contracted index. append this contraction to each of the sets of contractions output 
    # by generate_lorentz_contractions() acting on the decremented lorentzRanks_list. 
    
    # base case: if the total number of free Lorentz indices is 0 or 1, return empty list of contraction sets 
    if sum(lorentzRanks_list) < 2:
        contraction_list_list = [[]]
        return contraction_list_list
    
    # if sum of lorentz ranks is odd, there must be one uncontracted index remaining after all contractions. 
    # this free index can be associated with any field. for each field with non-zero lorentz rank, generate all 
    # contractions of fields where free index is attached to that field. do this by decrementing field rank by one
    # and then generating contractions for lorentz rank list with even total lorentz rank. 
    if sum(lorentzRanks_list) %2 != 0:
        N_fieldTypes = len(lorentzRanks_list)
        contraction_list_list = []
        for i in range(N_fieldTypes):
            if lorentzRanks_list[i] > 0:
                lorentzRanks_list_new = lorentzRanks_list.copy()
                lorentzRanks_list_new[i] -= 1
                contraction_list_list += generate_lorentz_contractions(lorentzRanks_list_new, 0)
        return contraction_list_list
                
    # otherwise, if sum of lorentzRanks_list is even, proceed as follows.
    
    # set to store different sets of contractions
    contraction_list_list = []
    # variable to store number of distinct fields
    N_fieldTypes = len(lorentzRanks_list)
    
    # find first field of non-zero rank for first index of contraction
    for i in range(N_fieldTypes):
        if lorentzRanks_list[i] >= 1:
            i_start = i
            lorentzRanks_list[i_start] -= 1
            #print('i_start: ' + str(i_start))
            break
            
    if i0 <= i_start:
        i0 = i_start
     
    # find all ways of contracting first non-zero lorentz rank field with other fields or itself 
    for i in range(i0, N_fieldTypes):
        #print(i)
        if lorentzRanks_list[i] >= 1:
            contraction = (i_start, i)
            
            lorentzRanks_list_old = lorentzRanks_list.copy()
            
            #decrement lorentzRanks_list_old 
            lorentzRanks_list_old[i] -= 1
            
            #if i_start of decremented list is the same as the non-decremented list, only include contractions
            #where second index is greater than or equal to i
            if lorentzRanks_list_old[i_start] > 0:
                contraction_list_list_old = generate_lorentz_contractions(lorentzRanks_list_old, i)
            else: 
                contraction_list_list_old = generate_lorentz_contractions(lorentzRanks_list_old, 0)
            
            for contraction_list_old in contraction_list_list_old:
                contraction_list = contraction_list_old.copy()
                contraction_list.append(contraction)
                contraction_list_list.append(contraction_list) 
                
    return contraction_list_list


def generate_lorentz_contractions_from_dict(field_dict):
    lorentzRanks = lorentzRanks_list(field_dict)
    contraction_list_list = generate_lorentz_contractions(lorentzRanks, 0)
    contraction_multisets_set = set()
    for item in contraction_list_list:
        contraction_multiset = FrozenMultiset(item)
        contraction_multisets_set.add(contraction_multiset)
    return contraction_multisets_set


In [7]:
from multiset import FrozenMultiset

field_dict1 = {'D': 4, 'F': 2, 'Pb_S_P': 0, 'Pb_V_P': 2, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
generate_lorentz_contractions_from_dict(field_dict1)

{FrozenMultiset({(3, 3): 1, (1, 1): 2, (0, 0): 2}),
 FrozenMultiset({(3, 3): 1, (1, 1): 1, (0, 1): 2, (0, 0): 1}),
 FrozenMultiset({(1, 1): 1, (0, 3): 2, (0, 1): 2}),
 FrozenMultiset({(1, 3): 2, (1, 1): 1, (0, 0): 2}),
 FrozenMultiset({(1, 3): 2, (0, 1): 2, (0, 0): 1}),
 FrozenMultiset({(1, 1): 2, (0, 3): 2, (0, 0): 1}),
 FrozenMultiset({(1, 3): 1, (0, 3): 1, (0, 1): 3}),
 FrozenMultiset({(1, 3): 1, (1, 1): 1, (0, 3): 1, (0, 1): 1, (0, 0): 1}),
 FrozenMultiset({(3, 3): 1, (0, 1): 4})}

# Generate all Field Combinations up to a Given Mass Dimension

In [8]:
def symbols_to_fields(field_combo_symbols):
    field_combo = []
    for field_symbol in field_combo_symbols:
        if field_symbol == 'D':
            field_combo.append(D)
            continue
        if field_symbol == 'F':
            field_combo.append(F)
            continue
        if field_symbol == 'Pb_S_P':
            field_combo.append(S)
            continue
        if field_symbol == 'Pb_V_P':
            field_combo.append(V)
            continue
        if field_symbol == 'Pb_T_P':
            field_combo.append(T)
            continue
        if field_symbol == 'Pb_Vp_P':
            field_combo.append(Vp)
            continue
        if field_symbol == 'Pb_Sp_P':
            field_combo.append(Sp)
            continue
         
    return field_combo

def totalMassDim(field_combo_symbols):
    field_combo = symbols_to_fields(field_combo_symbols)
    m = 0
    for field in field_combo:
        m += field.get_massDim()
    return m

def generate_field_combos(massDim):
    # EXPLANATION: Generate all multisets of the symbols [D, F, S, V, T, Vp, Sp] up to massDim. store in a set of 
    # multisets called set_of_field_multisets. generate new multisets by adding each field to multisets in a set
    # of multisets called front. remove a multiset from front after attempting to add each field to it. add an
    # element to front only if after a field has been added, its mass dimension is still less than massDim. returns
    # set_of_multisets, a dictionary mapping each mass dimension up to massDim to a set of multisets. 
    
    # declare field types
    D = field('D', 1, 1, 0, 0)
    F = field('F', 2, 2, 0, 0)
    S = field('Pb_S_P', 3, 0, 0, 0)
    V = field('Pb_V_P', 3, 0, 0, 0)
    T = field('Pb_T_P', 3, 0, 0, 0)
    Vp = field('Pb_Vp_P', 3, 0, 0, 0)
    Sp = field('Pb_Sp_P', 3, 0, 0, 0)
    
    # make list of field types
    fields = [D, F, S, V, T, Vp, Sp] 
    
    # set of multisets
    set_of_field_multisets = {}
    for m in range(massDim + 1): #set of multisets of field strings, arranged by massDim
        set_of_field_multisets[m] = set()
    front = set() #set of multisets of field strings, set of multisets to add to to make new multisets
    
    # initialize front
    for item in fields: 
        m = item.get_massDim()
        if m <= massDim:
            front.add(FrozenMultiset([item.get_symbol()]))
            set_of_field_multisets[m].add(FrozenMultiset([item.get_symbol()]))

    # build set_of_multisets
    while front:
        #print('')
        #print('front: ' + str(front))
        front_new = front.copy()
        for field_multiset in front:
            #print('field_multiset: ' + str(field_multiset))
            for item in fields:
                #print('field item: ' + str(item.get_symbol()))
                field_multiset_new = field_multiset.combine(FrozenMultiset([item.get_symbol()]))
                #print('field_multiset_new: ' + str(field_multiset_new))
                m = totalMassDim(field_multiset_new)
                if m < massDim:
                    set_of_field_multisets[m].add(field_multiset_new)
                    front_new.add(field_multiset_new)
                    #print('ADD TO FRONT: ' + str(field_multiset_new))
                if m == massDim:
                    set_of_field_multisets[m].add(field_multiset_new)
            front_new.remove(field_multiset)
        front = front_new
        #print('front_new: ' + str(front_new))
        #print('')
    
    return set_of_field_multisets


In [9]:
massDim = 5
for key in generate_field_combos(massDim):
    print(str(key) + ': ' + str(generate_field_combos(massDim)[key]))
    print('')

0: set()

1: {FrozenMultiset({'D': 1})}

2: {FrozenMultiset({'D': 2}), FrozenMultiset({'F': 1})}

3: {FrozenMultiset({'F': 1, 'D': 1}), FrozenMultiset({'Pb_T_P': 1}), FrozenMultiset({'Pb_Vp_P': 1}), FrozenMultiset({'Pb_S_P': 1}), FrozenMultiset({'Pb_V_P': 1}), FrozenMultiset({'Pb_Sp_P': 1}), FrozenMultiset({'D': 3})}

4: {FrozenMultiset({'Pb_V_P': 1, 'D': 1}), FrozenMultiset({'Pb_S_P': 1, 'D': 1}), FrozenMultiset({'F': 2}), FrozenMultiset({'F': 1, 'D': 2}), FrozenMultiset({'D': 4}), FrozenMultiset({'Pb_Sp_P': 1, 'D': 1}), FrozenMultiset({'Pb_T_P': 1, 'D': 1}), FrozenMultiset({'Pb_Vp_P': 1, 'D': 1})}

5: {FrozenMultiset({'Pb_V_P': 1, 'F': 1}), FrozenMultiset({'D': 5}), FrozenMultiset({'Pb_S_P': 1, 'D': 2}), FrozenMultiset({'Pb_V_P': 1, 'D': 2}), FrozenMultiset({'Pb_S_P': 1, 'F': 1}), FrozenMultiset({'Pb_Vp_P': 1, 'D': 2}), FrozenMultiset({'F': 1, 'D': 3}), FrozenMultiset({'Pb_Vp_P': 1, 'F': 1}), FrozenMultiset({'Pb_T_P': 1, 'D': 2}), FrozenMultiset({'Pb_T_P': 1, 'F': 1}), FrozenMultiset

Tested for massdim up to 5

Convert field multiset to field dictionary.

In [10]:
def convert_field_multiset_to_dict(field_multiset):
    field_dict = {'D': 0, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
    tmp_dict = dict(field_multiset.items())
    for key in tmp_dict.keys():
        field_dict[key] = tmp_dict[key]
    return field_dict

In [11]:
for field_multiset in generate_field_combos(massDim)[1]:
    print(convert_field_multiset_to_dict(field_multiset))

{'D': 1, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}


# Generate All Lorentz Contractions for Every Field Combination up to a Given Mass Dimension

In [12]:
def generate_lorentz_contractions_from_massDim(massDim, up_to = True):
    lorentz_contractions_dict = {}
    for m in range(1,massDim+1):
        #print('m: ' + str(m))
        lorentz_contractions_dict_m = {}
        field_combos_m = generate_field_combos(massDim)[m]
        for field_multiset in generate_field_combos(massDim)[m]:
            field_combo_dict = convert_field_multiset_to_dict(field_multiset)
            #print('field_combo_dict: ' + str(field_combo_dict))
            lorentz_contractions_dict_m[field_multiset] = generate_lorentz_contractions_from_dict(field_combo_dict)
            #print('generate_lorentz_contractions_reduced(field_combo_dict): ' + str(generate_lorentz_contractions_reduced(field_combo_dict)))
        lorentz_contractions_dict[m] = lorentz_contractions_dict_m
        #print('')
    if up_to == True:
        return lorentz_contractions_dict
    else: 
        return lorentz_contractions_dict[massDim]

In [13]:
# test with up_to = True
massDim = 4
lorentz_contraction_dict = generate_lorentz_contractions_from_massDim(massDim, up_to = True)
for massDim in lorentz_contraction_dict.keys():
    print(massDim)
    for field_combo in lorentz_contraction_dict[massDim].keys():
        print(convert_field_multiset_to_dict(field_combo))
        print(lorentz_contraction_dict[massDim][field_combo])
    print('')

  

1
{'D': 1, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({})}

2
{'D': 2, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({(0, 0): 1})}
{'D': 0, 'F': 1, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({(1, 1): 1})}

3
{'D': 1, 'F': 1, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({(0, 1): 1}), FrozenMultiset({(1, 1): 1})}
{'D': 0, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 1, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({(4, 4): 1})}
{'D': 0, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 1, 'Pb_Sp_P': 0}
{FrozenMultiset({})}
{'D': 0, 'F': 0, 'Pb_S_P': 1, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({})}
{'D': 0, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 1, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({})}
{'D': 0, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 1}
{Fr

See how high massDim can go.

In [14]:
# test with up_to = False
import time
start_time = time.time()

massDim = 6
lorentz_contraction_dict = generate_lorentz_contractions_from_massDim(massDim, up_to = False)

for field_combo in lorentz_contraction_dict.keys():
    print(convert_field_multiset_to_dict(field_combo))
    print(lorentz_contraction_dict[field_combo])
    print('')
    
print("--- %s seconds ---" % (time.time() - start_time))

{'D': 3, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 1}
{FrozenMultiset({(0, 0): 1})}

{'D': 0, 'F': 0, 'Pb_S_P': 1, 'Pb_V_P': 0, 'Pb_T_P': 1, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({(4, 4): 1})}

{'D': 0, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 1, 'Pb_Sp_P': 1}
{FrozenMultiset({})}

{'D': 3, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 1, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({(0, 4): 1, (0, 0): 1}), FrozenMultiset({(0, 4): 2}), FrozenMultiset({(4, 4): 1, (0, 0): 1})}

{'D': 1, 'F': 1, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 1, 'Pb_Vp_P': 0, 'Pb_Sp_P': 0}
{FrozenMultiset({(4, 4): 1, (1, 1): 1}), FrozenMultiset({(1, 4): 2}), FrozenMultiset({(1, 1): 1, (0, 4): 1}), FrozenMultiset({(4, 4): 1, (0, 1): 1}), FrozenMultiset({(1, 4): 1, (0, 1): 1}), FrozenMultiset({(1, 4): 1, (0, 4): 1})}

{'D': 0, 'F': 0, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 0, 'Pb_Vp_P': 0, 'Pb_Sp_P': 2}
{FrozenMultiset({})}

{'D': 0, 'F': 0, 'Pb_S_P': 1, 'Pb_V_P': 1, 'Pb_T

0.03 seconds for massDim 6. 

.95 seconds for massDim 12.

5.26 seconds for massDim 16.

20.6 seconds for massDim 20

In [15]:
start_time = time.time()

massDim = 22
lorentz_contraction_dict = generate_lorentz_contractions_from_massDim(massDim, up_to = False)

for field_combo in lorentz_contraction_dict.keys():
    print(convert_field_multiset_to_dict(field_combo))
    print(lorentz_contraction_dict[field_combo])
    print('')
    
print("--- %s seconds ---" % (time.time() - start_time))

{'D': 0, 'F': 2, 'Pb_S_P': 1, 'Pb_V_P': 1, 'Pb_T_P': 0, 'Pb_Vp_P': 3, 'Pb_Sp_P': 1}
{FrozenMultiset({(1, 5): 3, (1, 3): 1}), FrozenMultiset({(5, 5): 1, (1, 5): 1, (1, 3): 1, (1, 1): 1}), FrozenMultiset({(3, 5): 1, (1, 5): 2, (1, 1): 1}), FrozenMultiset({(5, 5): 1, (3, 5): 1, (1, 1): 2})}

{'D': 7, 'F': 0, 'Pb_S_P': 2, 'Pb_V_P': 1, 'Pb_T_P': 1, 'Pb_Vp_P': 0, 'Pb_Sp_P': 1}
{FrozenMultiset({(0, 4): 2, (0, 3): 1, (0, 0): 2}), FrozenMultiset({(3, 4): 1, (0, 4): 1, (0, 0): 3}), FrozenMultiset({(4, 4): 1, (0, 3): 1, (0, 0): 3})}

{'D': 0, 'F': 2, 'Pb_S_P': 1, 'Pb_V_P': 1, 'Pb_T_P': 1, 'Pb_Vp_P': 3, 'Pb_Sp_P': 0}
{FrozenMultiset({(5, 5): 1, (3, 4): 1, (1, 5): 1, (1, 4): 1, (1, 1): 1}), FrozenMultiset({(5, 5): 1, (1, 5): 1, (1, 4): 2, (1, 3): 1}), FrozenMultiset({(5, 5): 1, (4, 4): 1, (3, 5): 1, (1, 1): 2}), FrozenMultiset({(4, 5): 1, (3, 5): 1, (1, 5): 1, (1, 4): 1, (1, 1): 1}), FrozenMultiset({(5, 5): 1, (4, 5): 1, (1, 4): 1, (1, 3): 1, (1, 1): 1}), FrozenMultiset({(4, 4): 1, (1, 5): 3, (1, 3

{'D': 0, 'F': 2, 'Pb_S_P': 0, 'Pb_V_P': 3, 'Pb_T_P': 2, 'Pb_Vp_P': 1, 'Pb_Sp_P': 0}
{FrozenMultiset({(4, 5): 1, (4, 4): 1, (3, 4): 1, (1, 3): 2, (1, 1): 1}), FrozenMultiset({(4, 4): 2, (3, 5): 1, (3, 3): 1, (1, 1): 2}), FrozenMultiset({(4, 4): 2, (3, 3): 1, (1, 5): 1, (1, 3): 1, (1, 1): 1}), FrozenMultiset({(4, 4): 1, (3, 5): 1, (3, 3): 1, (1, 4): 2, (1, 1): 1}), FrozenMultiset({(4, 4): 2, (3, 5): 1, (1, 3): 2, (1, 1): 1}), FrozenMultiset({(4, 4): 1, (3, 5): 1, (3, 4): 2, (1, 1): 2}), FrozenMultiset({(4, 4): 1, (3, 5): 1, (3, 4): 1, (1, 4): 1, (1, 3): 1, (1, 1): 1}), FrozenMultiset({(4, 4): 1, (3, 5): 1, (1, 4): 2, (1, 3): 2}), FrozenMultiset({(3, 4): 2, (1, 5): 1, (1, 4): 2, (1, 3): 1}), FrozenMultiset({(3, 4): 1, (3, 3): 1, (1, 5): 1, (1, 4): 3}), FrozenMultiset({(3, 5): 1, (3, 4): 1, (1, 4): 3, (1, 3): 1}), FrozenMultiset({(4, 5): 1, (4, 4): 1, (1, 4): 1, (1, 3): 3}), FrozenMultiset({(3, 5): 1, (3, 4): 2, (1, 4): 2, (1, 1): 1}), FrozenMultiset({(3, 5): 1, (3, 3): 1, (1, 4): 4}), Fro

{FrozenMultiset({(1, 5): 1, (0, 5): 1, (0, 1): 7}), FrozenMultiset({(1, 5): 2, (1, 1): 1, (0, 1): 4, (0, 0): 2}), FrozenMultiset({(1, 5): 2, (0, 1): 6, (0, 0): 1}), FrozenMultiset({(1, 5): 1, (1, 1): 1, (0, 5): 1, (0, 1): 5, (0, 0): 1}), FrozenMultiset({(1, 1): 2, (0, 5): 2, (0, 1): 4, (0, 0): 1}), FrozenMultiset({(1, 1): 1, (0, 5): 2, (0, 1): 6}), FrozenMultiset({(1, 5): 2, (1, 1): 3, (0, 0): 4}), FrozenMultiset({(5, 5): 1, (1, 1): 3, (0, 1): 2, (0, 0): 3}), FrozenMultiset({(5, 5): 1, (0, 1): 8}), FrozenMultiset({(1, 1): 3, (0, 5): 2, (0, 1): 2, (0, 0): 2}), FrozenMultiset({(5, 5): 1, (1, 1): 1, (0, 1): 6, (0, 0): 1}), FrozenMultiset({(1, 1): 4, (0, 5): 2, (0, 0): 3}), FrozenMultiset({(1, 5): 2, (1, 1): 2, (0, 1): 2, (0, 0): 3}), FrozenMultiset({(5, 5): 1, (1, 1): 4, (0, 0): 4}), FrozenMultiset({(1, 5): 1, (1, 1): 3, (0, 5): 1, (0, 1): 1, (0, 0): 3}), FrozenMultiset({(1, 5): 1, (1, 1): 2, (0, 5): 1, (0, 1): 3, (0, 0): 2}), FrozenMultiset({(5, 5): 1, (1, 1): 2, (0, 1): 4, (0, 0): 2})}


{FrozenMultiset({(3, 3): 1, (1, 5): 1, (1, 3): 1, (1, 1): 1}), FrozenMultiset({(3, 5): 1, (1, 3): 2, (1, 1): 1}), FrozenMultiset({(1, 5): 1, (1, 3): 3}), FrozenMultiset({(3, 5): 1, (3, 3): 1, (1, 1): 2})}

{'D': 6, 'F': 2, 'Pb_S_P': 0, 'Pb_V_P': 0, 'Pb_T_P': 1, 'Pb_Vp_P': 2, 'Pb_Sp_P': 1}
{FrozenMultiset({(4, 5): 1, (1, 4): 1, (1, 1): 1, (0, 5): 1, (0, 1): 1, (0, 0): 2}), FrozenMultiset({(5, 5): 1, (1, 4): 2, (0, 1): 2, (0, 0): 2}), FrozenMultiset({(4, 4): 1, (1, 5): 2, (1, 1): 1, (0, 0): 3}), FrozenMultiset({(4, 4): 1, (1, 1): 2, (0, 5): 2, (0, 0): 2}), FrozenMultiset({(5, 5): 1, (4, 4): 1, (0, 1): 4, (0, 0): 1}), FrozenMultiset({(5, 5): 1, (1, 1): 1, (0, 4): 2, (0, 1): 2, (0, 0): 1}), FrozenMultiset({(4, 4): 1, (1, 5): 1, (1, 1): 1, (0, 5): 1, (0, 1): 1, (0, 0): 2}), FrozenMultiset({(1, 5): 1, (1, 4): 1, (1, 1): 1, (0, 5): 1, (0, 4): 1, (0, 0): 2}), FrozenMultiset({(4, 4): 1, (0, 5): 2, (0, 1): 4}), FrozenMultiset({(4, 5): 1, (1, 1): 1, (0, 5): 1, (0, 4): 1, (0, 1): 2, (0, 0): 1}), F

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Filter out Terms with Self Contractions on Anti-Symmetric Fields, and Terms with Only Derivatives

In [1]:
import copy

# for each field combo: if there is only one F in that combo, remove any multiset of contractions containing the 
# contraction (1,1); if there is only one T, remove any multiset of contractions containing the contraction (4,4).
# these correspond to terms in which an antisymmetric two-index field is contracted with itself, making the term
# zero. if there is more than one F or T the contractions need not be on the same F or T and so need not be zero.
def filter_terms(massDim, up_to = True):
    lorentz_contractions_dict = generate_lorentz_contractions_from_massDim(massDim, up_to = True)
    lorentz_contractions_dict_filtered = copy.deepcopy(lorentz_contractions_dict)
    # remove term containing only derivatives and no fields
    del lorentz_contractions_dict_filtered[massDim][FrozenMultiset({'D': 6})]
    # remove terms with self contractions on antisymmetric fields
    for m in lorentz_contractions_dict.keys():
        for field_combo in lorentz_contractions_dict[m].keys():
            for contraction_multiset in lorentz_contractions_dict[m][field_combo]:
                contraction_multiset_dict = dict(contraction_multiset.items())
                if (1,1) in contraction_multiset_dict.keys() and contraction_multiset_dict[(1,1)]==1:
                    lorentz_contractions_dict_filtered[m][field_combo].remove(contraction_multiset)
                    continue
                if (4,4) in contraction_multiset_dict.keys() and contraction_multiset_dict[(4,4)]==1:
                    lorentz_contractions_dict_filtered[m][field_combo].remove(contraction_multiset)
                    continue
    if up_to == True:
        return lorentz_contractions_dict_filtered
    else:
        return lorentz_contractions_dict_filtered[massDim]    

In [115]:
massDim = 6
filter_terms(massDim, up_to = False)

{FrozenMultiset({'D': 3, 'Pb_Sp_P': 1}): {FrozenMultiset({(0, 0): 1})},
 FrozenMultiset({'Pb_Sp_P': 2}): {FrozenMultiset({})},
 FrozenMultiset({'D': 1, 'Pb_Sp_P': 1, 'F': 1}): {FrozenMultiset({(0, 1): 1})},
 FrozenMultiset({'D': 3, 'Pb_S_P': 1}): {FrozenMultiset({(0, 0): 1})},
 FrozenMultiset({'Pb_T_P': 2}): {FrozenMultiset({(4, 4): 2})},
 FrozenMultiset({'Pb_Vp_P': 1, 'Pb_Sp_P': 1}): {FrozenMultiset({})},
 FrozenMultiset({'Pb_T_P': 1, 'Pb_S_P': 1}): set(),
 FrozenMultiset({'Pb_V_P': 1, 'Pb_Vp_P': 1}): {FrozenMultiset({(3, 5): 1})},
 FrozenMultiset({'D': 3, 'Pb_V_P': 1}): {FrozenMultiset({(0, 3): 1, (0, 0): 1})},
 FrozenMultiset({'Pb_Vp_P': 2}): {FrozenMultiset({(5, 5): 1})},
 FrozenMultiset({'D': 2, 'F': 2}): {FrozenMultiset({(1, 1): 2, (0, 0): 1})},
 FrozenMultiset({'D': 3, 'Pb_Vp_P': 1}): {FrozenMultiset({(0, 5): 1, (0, 0): 1})},
 FrozenMultiset({'Pb_T_P': 1, 'Pb_Sp_P': 1}): set(),
 FrozenMultiset({'Pb_T_P': 1, 'Pb_Vp_P': 1}): {FrozenMultiset({(4, 5): 1})},
 FrozenMultiset({'F': 3})

## Distribute Derivatives among Fields; Impose Integration by Parts (IBP)

Suppose we have the following combination of fields and derivatives:

\begin{align}
 D^{n_0} \ F^{n_1} \ \bar{\psi} S \psi^{n_2} \ \bar{\psi}V\psi^{n_3} \ \bar{\psi}T \psi^{n_4}  \ 
 \bar{\psi}V_{p}\psi^{n_5} \ \bar{\psi}S_{p} \psi^{n_6}
\end{align}

Each term generated from this combination of fields assumes a different way of distributing derivatives among the various $F$, $\psi$ and $\bar{\psi}$. It turns out because of the IBP relations that all such ways of distributing the derivatives among the fields are equivalent, so that we can eliminate all but one of the terms generated in this manner - for convenience, let's choose the term with all $n_0$ derivatives acting on a single $F$ field, and no derivatives acting on any of the other fields (we could also have chosen any other distribution of derivatives among the fields instead, since there are $n_0$ independent IBP relations and $n_0 + 1$ to distribute the derivatives between a single $F$ and all other fields). Thus, terms with the same fields and contractions but different distributions of derivatives are equivalent. Nothing needs to be done here if we interpret the terms generated above with the understanding that the derivatives may be distributed however we please. 

## Equations of Motion

Equations of motion for QED take the form 

\begin{align}
i \gamma^{\mu} D_{\mu} \psi &= m \psi \\
-i \gamma^{\mu} D_{\mu} \bar{\psi} &= m \bar{\psi} \\
D_{\mu} F^{\mu \nu} &= e \bar{\psi}\gamma^{\nu} \psi
\end{align}

We can use these equations to replace every $D_{\mu} \psi$ with a $\psi$ and every $D_{\mu} F^{\mu \nu}$ with a $\bar{\psi}\gamma^{\nu} \psi$. This enables us to completely remove all derivatives from the Lagrangian. If we see a term with a $D_{\mu} F^{\mu \nu}$ or $D_{\mu} \psi$, we can simply remove it. In principle, we can use IBP to shuffle all derivatives onto a single field, and then use just one of the EOM to eliminate all terms with derivatives. 

## Commutator of Gauge Covariant Derivatives

Since 

\begin{align}
[D_{\mu}, D_{\nu}] \sim F_{\mu \nu}
\end{align}

$D_{\mu} D_{\nu} F^{\mu \nu} \sim F_{\mu \nu} F^{\mu \nu}$ and $D_{\mu} D_{\nu} T^{\mu \nu} \sim F_{\mu \nu} T^{\mu \nu}$. So we can remove any term containing $D_{\mu} D_{\nu}$ contracted into $F^{\mu \nu}$ or $T^{\mu \nu}$.


# Function to Filter Redundant Terms

The user may input which fields should have derivatives. 

In [ ]:
def filter_terms_IBP_EOM_DD():
    return 